In [ ]:
from einshape import jax_einshape as einshape
from jax import lax
import jax
from functools import partial
import sys
sys.path.append('../')
import haiku as hk
import matplotlib.pyplot as plt
import jax.numpy as jnp
import numpy as np

from functools import partial
from jax import lax
import os

# Poisson equation

## Utils -- k有2个value，只适用于forward problem+inverse problem

In [ ]:


@partial(jax.jit, static_argnames=('num','kernel'))
def generate_gaussian_process(key, ts, num, kernel, k_sigma, k_l):
  '''
  ts: 1D array (length,)
  out: Gaussian process samples, 2D array (num, length)
  '''
  length = len(ts)
  mean = jnp.zeros((num,length))
  # cov = rbf_kernel(ts[:, None], ts[:, None], sigma=k_sigma, l=k_l)
  cov = kernel(ts, ts, sigma=k_sigma, l=k_l)
  cov = einshape('ii->nii', cov, n = num)
  out = jax.random.multivariate_normal(key, mean=mean, cov=cov, shape=(num,), method='svd')
  return out


def rbf_kernel_jax(x1, x2, sigma, l): # Define the covariance function
    """
    Radial basis function kernel, only support 1D x1 and x2
    """
    xx1, xx2 = jnp.meshgrid(x1, x2, indexing='ij')
    sq_norm = (xx1-xx2)**2/(l**2)
    return sigma**2 * jnp.exp(-0.5 * sq_norm)


def tridiagonal_solve(dl, d, du, b):
  """Pure JAX implementation of `tridiagonal_solve`."""
  prepend_zero = lambda x: jnp.append(jnp.zeros([1], dtype=x.dtype), x[:-1])
  fwd1 = lambda tu_, x: x[1] / (x[0] - x[2] * tu_)
  fwd2 = lambda b_, x: (x[0] - x[3] * b_) / (x[1] - x[3] * x[2])
  bwd1 = lambda x_, x: x[0] - x[1] * x_
  double = lambda f, args: (f(*args), f(*args))

  # Forward pass.
  _, tu_ = lax.scan(lambda tu_, x: double(fwd1, (tu_, x)),
                    du[0] / d[0],
                    (d, du, dl),
                    unroll=32)

  _, b_ = lax.scan(lambda b_, x: double(fwd2, (b_, x)),
                  b[0] / d[0],
                  (b, d, prepend_zero(tu_), dl),
                  unroll=32)

  # Backsubstitution.
  _, x_ = lax.scan(lambda x_, x: double(bwd1, (x_, x)),
                  b_[-1],
                  (b_[::-1], tu_[::-1]),
                  unroll=32)

  return x_[::-1]


@partial(jax.jit, static_argnames=("N"))
def solve_poisson(L, N, u_left, u_right, c): # Poisson equation solver
    '''
    du/dxx = c over domain [0,L]
    c: spatially varying function, size N-1,
    u_left, u_right: boundary conditions.
    the output is the full solution, (N+1) grid point values.
    '''
    dx = L / N
    # x = jnp.linspace(0, L, N+1)

    # finite difference matrix
    du = jnp.array([1.0] * (N-2) + [0.0])
    dl =  jnp.array([0.0] + [1.0] * (N-2))
    d = - 2.0 * jnp.ones((N-1,))

    b = c*dx*dx
    b = b.at[0].add(-u_left)
    b = b.at[-1].add(-u_right)

    out_u = tridiagonal_solve(dl, d, du, b)
    u = jnp.pad(out_u, (1, 1), mode='constant', constant_values=(u_left, u_right))
    return u

solve_poisson_batch = jax.jit(jax.vmap(solve_poisson, in_axes=(None, None, None, None, 0)), static_argnums=(1,))



def generate_pde_poisson(seed, eqns, length, dx, num, file_name, ood_type='all'):
  '''
  du/dxx = c(x) over domain [0,L]
  c(x) : spatially varying coefficient, size N-1,
          we use GP to sample c(x)
  u_left, u_right: boundary conditions.
  the output is the full solution, (N+1) grid point values.
  '''
  N = length
  L = length * dx
  rng = hk.PRNGSequence(jax.random.PRNGKey(seed))
  if ood_type == 'boundary' or ood_type == 'all':
        coeffs_ul = jax.random.uniform(next(rng), (eqns,), minval = -1.5, maxval = 1.5)  # 扩大范围
        coeffs_ur = jax.random.uniform(next(rng), (eqns,), minval = -1.5, maxval = 1.5)
  else:
        coeffs_ul = jax.random.uniform(next(rng), (eqns,), minval = -1.0, maxval = 1.0)
        coeffs_ur = jax.random.uniform(next(rng), (eqns,), minval = -1.0, maxval = 1.0)
    
  all_xs = []; all_cs = []; all_us = []

  """
  This is a 5D array. [eqns, num, 2, N+1, 2]
  First dimension: number of operators (or equations) 
  Second dimension: number of (u, c) pairs chosen for each operator
  Third dimension: index 0 indicates u; index 1 indicates c
  Fourth dimension: number of positions where x,u,c values are saved for each particular equation
  Fifth dimension: index 0 indicates x; index 1 indicates u or c value at the specified position
  """

  index = -1
  for i, (coeff_ul, coeff_ur) in enumerate(zip(coeffs_ul, coeffs_ur)):
    '''xs = jnp.linspace(0.0, L, N+1)# (N+1,)
    cs = generate_gaussian_process(next(rng), xs, num, kernel = rbf_kernel_jax, k_sigma = 2.0, k_l = 0.5) # (num, N+1)
    us = solve_poisson_batch(L, N, coeff_ul, coeff_ur, cs[:,1:-1]) # (num, N+1)
    all_xs.append(einshape("i->jik", xs, j = num, k = 1)) # (num, N+1, 1)
    all_cs.append(einshape("ij->ijk", cs, k = 1)) # (num, N+1, 1)
    all_us.append(einshape("ij->ijk", us, k = 1)) # (num, N+1, 1)'''
    xs = jnp.linspace(0.0, L, N+1)# (N+1,)
    if ood_type == 'gp' or ood_type == 'all':
        k_sigma = 4.0  # 增大方差
        k_l = 0.5    # 减小长度尺度，使函数变化更剧烈
    else:
        k_sigma = 2.0
        k_l = 0.5
    cs = generate_gaussian_process(next(rng), xs, num, rbf_kernel_jax, k_sigma, k_l) # (num, N+1)
    us = solve_poisson_batch(L, N, coeff_ul, coeff_ur, cs[:,1:-1]) # (num, N+1)
    all_xs.append(einshape("i->jik", xs, j = num, k = 1)) # (num, N+1, 1)
    
    # 为c添加标识符1
    cs_with_id = np.zeros((cs.shape[0], cs.shape[1], 2))  # (num, N+1, 2)
    cs_with_id[:,:,0] = cs
    cs_with_id[:,:,1] = 1  # 标识这是c
    all_cs.append(cs_with_id)
    
    # 为u添加标识符0
    us_with_id = np.zeros((us.shape[0], us.shape[1], 2))  # (num, N+1, 2)
    us_with_id[:,:,0] = us
    us_with_id[:,:,1] = 0  # 标识这是u
    all_us.append(us_with_id)

  all_xs = np.array(all_xs) # (eqns, num, N+1, 1)
  all_cs = np.array(all_cs) # (eqns, num, N+1, 1)
  all_us = np.array(all_us) # (eqns, num, N+1, 1)

  all_u = np.concatenate([all_xs, all_us], axis = -1) # (eqns, num, N+1, 2)
  all_c = np.concatenate([all_xs, all_cs], axis = -1) # (eqns, num, N+1, 2)
  data = np.concatenate([all_u[:,:,None,:,:], all_c[:,:,None,:,:]], axis = 2) # (eqns, num, 2, N+1, 2)
  root_dir = '/home/wanghanyang.01/ICON-Feature-Learning/data_generation/data'
  os.mkdir(f'{root_dir}/data_possion_jax')
  file_path = f'{root_dir}/data_possion_jax/{file_name}.npy'
  np.save(file_path, data)
  print("Data saved successfully")


## Utils -- k有1个value，只适用于forward problem

In [ ]:


@partial(jax.jit, static_argnames=('num','kernel'))
def generate_gaussian_process(key, ts, num, kernel, k_sigma, k_l):
  '''
  ts: 1D array (length,)
  out: Gaussian process samples, 2D array (num, length)
  '''
  length = len(ts)
  mean = jnp.zeros((num,length))
  # cov = rbf_kernel(ts[:, None], ts[:, None], sigma=k_sigma, l=k_l)
  cov = kernel(ts, ts, sigma=k_sigma, l=k_l)
  cov = einshape('ii->nii', cov, n = num)
  out = jax.random.multivariate_normal(key, mean=mean, cov=cov, shape=(num,), method='svd')
  return out


def rbf_kernel_jax(x1, x2, sigma, l): # Define the covariance function
    """
    Radial basis function kernel, only support 1D x1 and x2
    """
    xx1, xx2 = jnp.meshgrid(x1, x2, indexing='ij')
    sq_norm = (xx1-xx2)**2/(l**2)
    return sigma**2 * jnp.exp(-0.5 * sq_norm)


def tridiagonal_solve(dl, d, du, b):
  """Pure JAX implementation of `tridiagonal_solve`."""
  prepend_zero = lambda x: jnp.append(jnp.zeros([1], dtype=x.dtype), x[:-1])
  fwd1 = lambda tu_, x: x[1] / (x[0] - x[2] * tu_)
  fwd2 = lambda b_, x: (x[0] - x[3] * b_) / (x[1] - x[3] * x[2])
  bwd1 = lambda x_, x: x[0] - x[1] * x_
  double = lambda f, args: (f(*args), f(*args))

  # Forward pass.
  _, tu_ = lax.scan(lambda tu_, x: double(fwd1, (tu_, x)),
                    du[0] / d[0],
                    (d, du, dl),
                    unroll=32)

  _, b_ = lax.scan(lambda b_, x: double(fwd2, (b_, x)),
                  b[0] / d[0],
                  (b, d, prepend_zero(tu_), dl),
                  unroll=32)

  # Backsubstitution.
  _, x_ = lax.scan(lambda x_, x: double(bwd1, (x_, x)),
                  b_[-1],
                  (b_[::-1], tu_[::-1]),
                  unroll=32)

  return x_[::-1]


@partial(jax.jit, static_argnames=("N"))
def solve_poisson(L, N, u_left, u_right, c): # Poisson equation solver
    '''
    du/dxx = c over domain [0,L]
    c: spatially varying function, size N-1,
    u_left, u_right: boundary conditions.
    the output is the full solution, (N+1) grid point values.
    '''
    dx = L / N
    # x = jnp.linspace(0, L, N+1)

    # finite difference matrix
    du = jnp.array([1.0] * (N-2) + [0.0])
    dl =  jnp.array([0.0] + [1.0] * (N-2))
    d = - 2.0 * jnp.ones((N-1,))

    b = c*dx*dx
    b = b.at[0].add(-u_left)
    b = b.at[-1].add(-u_right)

    out_u = tridiagonal_solve(dl, d, du, b)
    u = jnp.pad(out_u, (1, 1), mode='constant', constant_values=(u_left, u_right))
    return u

solve_poisson_batch = jax.jit(jax.vmap(solve_poisson, in_axes=(None, None, None, None, 0)), static_argnums=(1,))



def generate_pde_poisson(seed, eqns, length, dx, num, file_name, ood_type='all'):
  '''
  du/dxx = c(x) over domain [0,L]
  c(x) : spatially varying coefficient, size N-1,
          we use GP to sample c(x)
  u_left, u_right: boundary conditions.
  the output is the full solution, (N+1) grid point values.
  '''
  N = length
  L = length * dx
  rng = hk.PRNGSequence(jax.random.PRNGKey(seed))
  if ood_type == 'boundary' or ood_type == 'all':
        coeffs_ul = jax.random.uniform(next(rng), (eqns,), minval = -2, maxval = 2)  # 扩大范围 +-1.5
        coeffs_ur = jax.random.uniform(next(rng), (eqns,), minval = -2, maxval = 2)  # +-2
  else:
        coeffs_ul = jax.random.uniform(next(rng), (eqns,), minval = -1.0, maxval = 1.0)
        coeffs_ur = jax.random.uniform(next(rng), (eqns,), minval = -1.0, maxval = 1.0)
    
  all_xs = []; all_cs = []; all_us = []

  """
  This is a 5D array. [eqns, num, 2, N+1, 2]
  First dimension: number of operators (or equations) 
  Second dimension: number of (u, c) pairs chosen for each operator
  Third dimension: index 0 indicates u; index 1 indicates c
  Fourth dimension: number of positions where x,u,c values are saved for each particular equation
  Fifth dimension: index 0 indicates x; index 1 indicates u or c value at the specified position
  """

  index = -1
  for i, (coeff_ul, coeff_ur) in enumerate(zip(coeffs_ul, coeffs_ur)):
    '''xs = jnp.linspace(0.0, L, N+1)# (N+1,)
    cs = generate_gaussian_process(next(rng), xs, num, kernel = rbf_kernel_jax, k_sigma = 2.0, k_l = 0.5) # (num, N+1)
    us = solve_poisson_batch(L, N, coeff_ul, coeff_ur, cs[:,1:-1]) # (num, N+1)
    all_xs.append(einshape("i->jik", xs, j = num, k = 1)) # (num, N+1, 1)
    all_cs.append(einshape("ij->ijk", cs, k = 1)) # (num, N+1, 1)
    all_us.append(einshape("ij->ijk", us, k = 1)) # (num, N+1, 1)'''
    xs = jnp.linspace(0.0, L, N+1)# (N+1,)
    if ood_type == 'gp' or ood_type == 'all':
        k_sigma = 3.0  # 增大方差 3.0
        k_l = 0.5   # 减小长度尺度，使函数变化更剧烈 0.5
    else:
        k_sigma = 2.0
        k_l = 0.5
    cs = generate_gaussian_process(next(rng), xs, num, rbf_kernel_jax, k_sigma, k_l) # (num, N+1)
    us = solve_poisson_batch(L, N, coeff_ul, coeff_ur, cs[:,1:-1]) # (num, N+1)
    all_xs.append(einshape("i->jik", xs, j = num, k = 1)) # (num, N+1, 1)
    
    # 为c添加标识符1
    cs_with_id = np.zeros((cs.shape[0], cs.shape[1], 1))  # (num, N+1, 2)
    cs_with_id[:,:,0] = cs
    all_cs.append(cs_with_id)
    
    # 为u添加标识符0
    us_with_id = np.zeros((us.shape[0], us.shape[1], 1))  # (num, N+1, 2)
    us_with_id[:,:,0] = us
    all_us.append(us_with_id)

  all_xs = np.array(all_xs) # (eqns, num, N+1, 1)
  all_cs = np.array(all_cs) # (eqns, num, N+1, 1)
  all_us = np.array(all_us) # (eqns, num, N+1, 1)

  all_u = np.concatenate([all_xs, all_us], axis = -1) # (eqns, num, N+1, 2)
  all_c = np.concatenate([all_xs, all_cs], axis = -1) # (eqns, num, N+1, 2)
  data = np.concatenate([all_u[:,:,None,:,:], all_c[:,:,None,:,:]], axis = 2) # (eqns, num, 2, N+1, 2)
  
  file_path = f'/home/wanghanyang.01/icon-feature_learning/scratch/data/data_possion_jax/{file_name}.npy'
  np.save(file_path, data)
  print("Data saved successfully")


## 生成模块

In [ ]:
valid_seed = 2  # 使用不同的随机种子
valid_eqns = 200  # 200个方程用于验证
valid_xs = 100  # 保持与训练集相同的空间离散化
valid_dx = 0.01
valid_num_ucpair = 100
valid_file_name = "Poisson_PDE_data_valid_1k_-15_405" # 1维，-2～1，3var，0.45

# 生成验证集
print("\nGenerating validation data...")
valid_data = generate_pde_poisson(
    seed=valid_seed,
    eqns=valid_eqns,
    length=valid_xs,
    dx=valid_dx,
    num=valid_num_ucpair,
    file_name=valid_file_name,
    ood_type='all'
)

In [ ]:
# 训练集参数
train_seed = 1
train_eqns = 1000  # 800个方程用于训练
train_xs = 100
train_dx = 0.01
train_num_ucpair = 100
train_file_name = "Poisson_PDE_data_train_2k"

# 生成训练集
print("Generating training data...")
train_data = generate_pde_poisson(
    seed=train_seed,
    eqns=train_eqns,
    length=train_xs,
    dx=train_dx,
    num=train_num_ucpair,
    file_name=train_file_name
)

# 验证集参数
valid_seed = 2  # 使用不同的随机种子
valid_eqns = 100  # 200个方程用于验证
valid_xs = train_xs  # 保持与训练集相同的空间离散化
valid_dx = train_dx
valid_num_ucpair = train_num_ucpair
valid_file_name = "Poisson_PDE_data_valid_2k"

# 生成验证集
print("\nGenerating validation data...")
valid_data = generate_pde_poisson(
    seed=valid_seed,
    eqns=valid_eqns,
    length=valid_xs,
    dx=valid_dx,
    num=valid_num_ucpair,
    file_name=valid_file_name
)


In [ ]:
data = np.load(f'/home/wanghanyang.01/icon-feature_learning/scratch/data/data_helmholtz_jax/Helmholtz_data_valid_1k.npy')
print(data.shape)

In [ ]:
for i in range(5):
  plt.figure()
  for j in range(5):
    plt.plot(data[i,j,0,:,0], data[i,j,0,:,1], label = "u", linestyle = "--")
    plt.plot(data[i,j,1,:,0], data[i,j,1,:,1], label = "c", linestyle = "-")
  plt.show()